# Data Preparation

The objective is to clean data, enrich the dataset, and load it to different tables in PostgreSQL database.

In [10]:
import pandas as pd
import psycopg2
import csv
import warnings

warnings.filterwarnings('ignore')

# Orders

In [21]:
df = pd.read_csv('orders.csv')

In [22]:
df['WEIGHT'] = df['QUANTITY']*24

In [23]:
df['AMOUNT'] = ''

In [24]:
prices = {'AJA001':1250, 'SOK001':1230, 'DUN001':1180, 'LOT001':1200, 'PEM001':1250,'BAB001':1210,'MAI001':1190, 'SMA001':1190, 'UMI001':1215}
df['AMOUNT'] = df['PRDTID'].map(prices) * df['QUANTITY']

In [25]:
empty_strings = df[df['AMOUNT'] == '']
print(empty_strings['AMOUNT'])

Series([], Name: AMOUNT, dtype: int64)


In [26]:
df['ORDTIME'] = pd.to_datetime(df['ORDTIME'], format='%H:%M:%S').dt.time
df['ORDDATE'] = pd.to_datetime(df['ORDDATE'], format='%y/%m/%d')

In [27]:
df['EVENTTIME'] = pd.to_datetime(df['ORDDATE'].astype(str) + ' ' + df['ORDTIME'].astype(str))

In [28]:
df.head()

,ID,CSMCODE,ORDDATE,ORDTIME,PRDTID,QUANTITY,WEIGHT,AMOUNT,EVENTTIME
0,10001,CSM001,2014-07-23,06:25:00,AJA001,10,240,12500,2014-07-23 06:25:00
1,10001,CSM001,2014-07-23,06:30:00,SOK001,10,240,12300,2014-07-23 06:30:00
2,10002,CSM003,2014-07-23,06:35:00,AJA001,25,600,31250,2014-07-23 06:35:00
3,10003,CSM002,2014-07-23,06:40:00,DUN001,10,240,11800,2014-07-23 06:40:00
4,10003,CSM002,2014-07-23,06:45:00,LOT001,20,480,24000,2014-07-23 06:45:00


In [29]:
df.dtypes

ID                    int64
CSMCODE              object
ORDDATE      datetime64[ns]
ORDTIME              object
PRDTID               object
QUANTITY              int64
WEIGHT                int64
AMOUNT                int64
EVENTTIME    datetime64[ns]
dtype: object

# Products

In [10]:
prd= df[['PRDTID']]
prd['NAME'], prd['STOCK'], prd['RESTOCK'], prd['UNITWEIGHT'] = '', '1500', '1000', '24'

In [11]:
prd['PRDTID'].unique()

array(['AJA001', 'SOK001', 'DUN001', 'LOT001', 'BAB001', 'MAI001',
       'SMA001', 'PEM001', 'UMI001'], dtype=object)

In [12]:
values = {'AJA001':'Ajab HB 2kg', 'SOK001':'Soko HB 2kg', 'DUN001':'Dunia HB 2kg', 'LOT001':'Lotus HB 2kg', 'PEM001':'Pembe HB 2kg','BAB001':'Baba Lao HB 2kg', 'SMA001':'Smatta HB 2kg', 'UMI001':'Umi HB 2kg'}
prd['NAME'] = prd['PRDTID'].map(values)

In [13]:
prd.rename(columns = {'PRDTID':'ID'}, inplace = True)

In [14]:
prd.head()

,ID,NAME,STOCK,RESTOCK,UNITWEIGHT
0,AJA001,Ajab HB 2kg,1500,1000,24
1,SOK001,Soko HB 2kg,1500,1000,24
2,AJA001,Ajab HB 2kg,1500,1000,24
3,DUN001,Dunia HB 2kg,1500,1000,24
4,LOT001,Lotus HB 2kg,1500,1000,24


In [37]:
prd.dtypes

ID            object
NAME          object
STOCK         object
RESTOCK       object
UNITWEIGHT    object
dtype: object

# Pricelist

In [39]:
prc= df[['PRDTID']]

In [40]:
prc['ID'], prc['PRICE'], prc['UOM'] = 'PRC1', '', 'Bale'

In [45]:
prices = {'AJA001':1250, 'SOK001':1230, 'DUN001':1180, 'LOT001':1200, 'PEM001':1250,'BAB001':1210, 'SMA001':1190, 'UMI001':1215, 'MAI001':1170}
prc['PRICE'] = prc['PRDTID'].map(prices)

In [46]:
new_index = ['ID', 'PRDTID', 'UOM', 'PRICE']
prc = prc.reindex(columns=new_index)

In [47]:
prc['PRICE'] = prc['PRICE'].astype('Int64')

In [48]:
prc.head(10)

,ID,PRDTID,UOM,PRICE
0,PRC1,AJA001,Bale,1250
1,PRC1,SOK001,Bale,1230
2,PRC1,AJA001,Bale,1250
3,PRC1,DUN001,Bale,1180
4,PRC1,LOT001,Bale,1200
5,PRC1,BAB001,Bale,1210
6,PRC1,MAI001,Bale,1170
7,PRC1,SMA001,Bale,1190
8,PRC1,AJA001,Bale,1250
9,PRC1,PEM001,Bale,1250


In [49]:
prc.dtypes

ID        object
PRDTID    object
UOM       object
PRICE      Int64
dtype: object

# Save

In [51]:
# The 'index=False 'parameter ensures that the index column is not included in the saved CSV file

df.to_csv('Orders.csv', index=False)
prd.to_csv('Products.csv', index=False)
prc.to_csv('Pricelist.csv', index=False)

In [30]:
df.to_csv('Orders.csv', index=False)

Load Orders Table

In [31]:
table_names = ['orders']
csv_files = ['Orders.csv']

try:
    with psycopg2.connect(
            host="localhost",
            port=5432,
            database="database",
            user="username",
            password="password"
    ) as conn, conn.cursor() as cursor:
        for table_name, csv_file in zip(table_names, csv_files):
            cursor.execute(
                "SELECT EXISTS (SELECT FROM information_schema.tables WHERE table_name = %s)",
                (table_name,)
            )
            table_exists = cursor.fetchone()[0]

            if not table_exists:
                cursor.execute("""
                    CREATE TABLE {} (
                        id int,
                        csmcode VARCHAR(10),
                        orddate DATE,
                        ordtime TIME,
                        prdtid VARCHAR(10),
                        quantity INTEGER,
                        weight INTEGER,
                        amount INTEGER,
                        eventtime TIME,
                    )
                """.format(table_name))
                print(f"Table '{table_name}' created successfully.")

            with open(csv_file, 'r') as file:
                next(file)
                cursor.copy_expert(
                "COPY {} FROM STDIN WITH CSV NULL 'NULL'".format(table_name),
                file)
                print(f"Data loaded into '{table_name}' table successfully.")

except psycopg2.Error as e:
    print(f"An error occurred: {e}")


Data loaded into 'orders' table successfully.


Load Products Table

In [33]:
table_names = ['products']
csv_files = ['Products.csv']

try:
    with psycopg2.connect(
            host="localhost",
            port=5432,
            database="database",
            user="username",
            password="password"
    ) as conn, conn.cursor() as cursor:
        for table_name, csv_file in zip(table_names, csv_files):
            cursor.execute(
                "SELECT EXISTS (SELECT FROM information_schema.tables WHERE table_name = %s)",
                (table_name,)
            )
            table_exists = cursor.fetchone()[0]

            if not table_exists:
                cursor.execute("""
                    CREATE TABLE {} (
                        id VARCHAR(10),
                        name VARCHAR(50),
                        stock INTEGER,
                        restock INTEGER,
                        unitweight INTEGER
                    )
                """.format(table_name))
                print(f"Table '{table_name}' created successfully.")

            with open(csv_file, 'r') as file:
                next(file)
                cursor.copy_expert(
                "COPY {} FROM STDIN WITH CSV NULL 'NULL'".format(table_name),
                file)
                print(f"Data loaded into '{table_name}' table successfully.")

except psycopg2.Error as e:
    print(f"An error occurred: {e}")

Data loaded into 'products' table successfully.


Load Pricelist Table

In [52]:
table_names = ['pricelist']
csv_files = ['Pricelist.csv']

try:
    with psycopg2.connect(
            host="localhost",
            port=5432,
            database="database",
            user="username",
            password="password"
    ) as conn, conn.cursor() as cursor:
        for table_name, csv_file in zip(table_names, csv_files):
            cursor.execute(
                "SELECT EXISTS (SELECT FROM information_schema.tables WHERE table_name = %s)",
                (table_name,)
            )
            table_exists = cursor.fetchone()[0]

            if not table_exists:
                cursor.execute("""
                    CREATE TABLE {} (
                        id VARCHAR(10),
                        prdtid VARCHAR(10), 
                        uom VARCHAR(10),
                        price INTEGER
                    )
                """.format(table_name))
                print(f"Table '{table_name}' created successfully.")

            with open(csv_file, 'r') as file:
                next(file)
                cursor.copy_expert(
                "COPY {} FROM STDIN WITH CSV NULL 'NULL'".format(table_name),
                file)
                print(f"Data loaded into '{table_name}' table successfully.")

except psycopg2.Error as e:
    print(f"An error occurred: {e}")

Data loaded into 'pricelist' table successfully.
